In [3]:
import pandas as pd
import numpy as np
#from nilearn.input_data import *
import ants
from matplotlib import pyplot as plt
import seaborn as sns
#(106, 51, 51, 51)

from sklearn.linear_model import *
from sklearn.multioutput import MultiOutputRegressor

import os

from datetime import datetime
now = datetime.now


In [4]:
nrois = 51

In [5]:
csv_fn = '/Users/aidasaglinskas/Desktop/subsFinal.csv'
df = pd.read_csv(csv_fn)
subs = np.array(df['BIDS ID'])
nsubs = len(subs)

In [6]:
#f = 0
#s = 0
#m = 1
#sub = subs[s]

In [7]:
# Get maskList
maskList = ['pmask_3001_{:05d}.nii'.format(i) for i in range(50+1)]
maskList[0] = 'imask.nii'
maskList
nmasks = len(maskList)

In [8]:
def getFNs(sub,f,m):
    root = '/Volumes/Samsung_T5/DS-use-new-atlas/'
    fileList = ['{}_task-face_run-1',
    '{}_task-lang_run-1',
    '{}_task-lang_run-2',
    '{}_task-rest_run-1',
    '{}_task-rest_run-2']

    mask_fn_temp = os.path.join(root,'{}/atlas/{}')
    data_fn_temp = os.path.join(root,'{}/func/rr{}_bold.nii')
    rp_fn_temp = os.path.join(root,'{}/func/rp_{}_bold.txt')

    mask_fn = mask_fn_temp.format(sub,maskList[m])
    data_fn = data_fn_temp.format(sub,fileList[f].format(sub))
    rp_fn = rp_fn_temp.format(sub,fileList[f].format(sub))
    
    return data_fn,mask_fn,rp_fn

In [9]:
# Gray Matter IDX
def get_GMidx(sub):
    data_fn,mask_fn,rp_fn = getFNs(sub,0,0)
    mask = ants.image_read(mask_fn)
    maskArr = mask.numpy()
    GM_idx = maskArr>=1
    return GM_idx

In [10]:
def get_data(data_fn,rp,GM_idx):
    
    data = ants.image_read(data_fn) # ANTS obj
    dataArr = data.numpy() # 4d arr
    dataFlat = dataArr[GM_idx] # 2d array
    
    # Regress out movement
    X = rp
    Y = dataFlat.transpose()
    clf = MultiOutputRegressor(LinearRegression(copy_X=True)).fit(X,Y)
    Y_hat = clf.predict(X)
    resid = Y-Y_hat
    
    return {'resid' : resid, 'Y_hat' : Y_hat, 'Y' : Y, 'X' : X}

In [11]:
def get_maskMat(sub,GM_idx):
    m = 0
    mask_fn = getFNs(sub,0,m)[1]
    mask = ants.image_read(mask_fn)
    maskArr = mask.numpy()
    maskMat = maskArr[GM_idx]

    for m in np.arange(1,nmasks):
        mask_fn = getFNs(sub,0,m)[1]
        mask = ants.image_read(mask_fn)
        maskArr = mask.numpy()
        #maskMat = maskArr[GM_idx]
        maskMat = np.vstack((maskMat,maskArr[GM_idx]))
    return maskMat

In [16]:
# f = 4
# s = 104
# m = 0
# sub = subs[s];sub

# data_fn,mask_fn,rp_fn = getFNs(sub,f,m)
# rp = np.array(pd.read_csv(rp_fn,sep='\s+',header=None))
# GM_idx = get_GMidx(sub)
# get_data(data_fn,rp,GM_idx)

In [19]:
subs

array(['sub-004', 'sub-012', 'sub-014', 'sub-018', 'sub-025', 'sub-029',
       'sub-030', 'sub-039', 'sub-042', 'sub-043', 'sub-044', 'sub-058',
       'sub-064', 'sub-066', 'sub-074', 'sub-076', 'sub-090', 'sub-091',
       'sub-097', 'sub-098', 'sub-101', 'sub-120', 'sub-122', 'sub-125',
       'sub-133', 'sub-142', 'sub-143', 'sub-144', 'sub-145', 'sub-148',
       'sub-153', 'sub-154', 'sub-155', 'sub-158', 'sub-159', 'sub-160',
       'sub-162', 'sub-163', 'sub-166', 'sub-167', 'sub-171', 'sub-175',
       'sub-177', 'sub-178', 'sub-181', 'sub-190', 'sub-191', 'sub-201',
       'sub-203', 'sub-204', 'sub-205', 'sub-207', 'sub-212', 'sub-226',
       'sub-227', 'sub-228', 'sub-229', 'sub-230', 'sub-237', 'sub-238',
       'sub-239', 'sub-240', 'sub-242', 'sub-244', 'sub-245', 'sub-252',
       'sub-253', 'sub-257', 'sub-258', 'sub-259', 'sub-262', 'sub-264',
       'sub-270', 'sub-272', 'sub-273', 'sub-274', 'sub-275', 'sub-276',
       'sub-279', 'sub-280', 'sub-284', 'sub-285', 

In [17]:
#overlap = np.array([sum(maskMat[0,:]==maskMat[r,:]) / maskMat.shape[1] for r in range(maskMat.shape[0])]);

In [18]:
f = 4
Biggie = np.zeros( (nsubs,nmasks,nrois,nrois) ) # Pre-allocate
print('started: {}'.format(now() ))
for s in np.arange(nsubs-1):
    print('{}/{}'.format(s,nsubs))
    sub = subs[s];sub
    data_fn,mask_fn,rp_fn = getFNs(sub,f,0)
    rp = np.array(pd.read_csv(rp_fn,sep='\s+',header=None))
    GM_idx = get_GMidx(sub)

    # Get Cleaned Data
    data = get_data(data_fn,rp,GM_idx)

    # Get all Masks
    maskMat = get_maskMat(sub,GM_idx)

    for m in np.arange(nmasks):
        parcelMat = np.array([data['resid'][:,maskMat[m,:]==r].mean(axis=1) for r in np.arange(1,nrois+1)])
        cmat = np.corrcoef(parcelMat)
        Biggie[s,m,:,:] = cmat

print('finished: {}'.format(now() ))

started: 2020-07-04 15:21:47.557491
0/106
1/106
2/106
3/106
4/106
5/106
6/106
7/106
8/106
9/106
10/106
11/106
12/106
13/106
14/106
15/106
16/106
17/106
18/106
19/106
20/106
21/106
22/106
23/106
24/106
25/106
26/106
27/106
28/106
29/106
30/106
31/106
32/106
33/106
34/106
35/106
36/106
37/106
38/106
39/106
40/106
41/106
42/106
43/106
44/106
45/106
46/106
47/106
48/106
49/106
50/106
51/106
52/106
53/106
54/106
55/106
56/106
57/106
58/106
59/106
60/106
61/106
62/106
63/106
64/106
65/106
66/106
67/106
68/106
69/106
70/106
71/106
72/106
73/106
74/106
75/106
76/106
77/106
78/106
79/106
80/106
81/106
82/106
83/106
84/106
85/106
86/106
87/106
88/106
89/106
90/106
91/106
92/106
93/106
94/106
95/106
96/106
97/106
98/106
99/106
100/106
101/106
102/106
103/106
104/106
finished: 2020-07-04 16:05:19.980511


In [25]:
Biggie[105,0,:,:]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
print(data_fn)

/Volumes/Samsung_T5/DS-use-new-atlas/sub-337/func/rrsub-337_task-rest_run-2_bold.nii


In [27]:
nm='task-rest_run-2'
ofn='/Users/aidasaglinskas/Desktop/{}'.format(nm)
print(ofn)

/Users/aidasaglinskas/Desktop/task-rest_run-2


In [28]:
np.save(ofn,Biggie)

In [ ]:

#ofn = '/Users/aidasaglinskas/Desktop/anatomical.npy'

In [ ]:
def flatten_C(C):
    nsubs = C.shape[0]
    ndim = C.shape[1]
    s = 0
    conn = C[s,:,:][np.triu_indices(ndim,1)]
    for s in range(1,nsubs):
        conn = np.vstack((conn,C[s,:,:][np.triu_indices(ndim,1)]))
    return conn

In [ ]:
# Sanity Checks
#overlap = np.array([sum(maskMat[0,:]==maskMat[r,:]) / maskMat.shape[1] for r in range(maskMat.shape[0])]);

In [ ]:
#plt.plot(overlap)
#sns.distplot(  np.array(flatten_C(Biggie[:,0,:,:])   -   flatten_C(Biggie[:,50,:,:]))  )